In [12]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv
import os

In [35]:
load_dotenv("../.env")

True

In [36]:
pine_cone_api_key = os.environ["PINECONE_API_KEY"]
openai_api_key = os.environ["OPENAI_API_KEY"]

In [54]:
PROMPT_TEMPLATE = """
    You are a smart saaistant who has a good understading about the postgres schemas and tables.

    Answer the question based only on the following context:
    
    Context:
    Below here are the schema information about two tables that is colleagues_data and asset_data. Based on the the question ask you need to detect
    whether which tables will be needed to make a sql query. 

    colleague_data schema:
    {colleague_data_schema}

    asset_data_schema:
    {asset_data_schema}


    Example1 Question:
    Show me the number of colleagues from India
    Example1 Answer:
    colleague_data to be used


    Example2 Question:
    Show me the number of colleagues working in Technology
    Example2 Answer:
    colleague_data to be used


    Example3 Question:
    Show me the details of the colleagues from India who are using macbook pro
    Example3 Answer:
    colleague_data and asset_data tables to be used
    
    ---

    Answer the question based on the above context: {question}
    You need to give me the names of tables in a list form
    also the columns that would be required
    """

In [55]:
colleague_data_schema = """
CREATE TABLE IF NOT EXISTS public.colleague_data
(
    employee_id integer NOT NULL,
    emp_name character varying COLLATE pg_catalog."default",
    work_location character varying COLLATE pg_catalog."default",
    work_country character varying COLLATE pg_catalog."default",
    business_unit character varying COLLATE pg_catalog."default",
    department character varying COLLATE pg_catalog."default",
    designation character varying COLLATE pg_catalog."default",
    band character varying COLLATE pg_catalog."default",
    role character varying COLLATE pg_catalog."default",
    date_of_joining character varying COLLATE pg_catalog."default",
    year_of_joining integer,
    joining_month character varying COLLATE pg_catalog."default",
    CONSTRAINT colleague_data_pkey PRIMARY KEY (employee_id)
)
"""


asset_data_schema = """
CREATE TABLE IF NOT EXISTS public.asset_data
(
    employee_id integer,
    laptop_model character varying COLLATE pg_catalog."default",
    manufacturer character varying COLLATE pg_catalog."default",
    CONSTRAINT fk FOREIGN KEY (employee_id)
        REFERENCES public.colleague_data (employee_id) MATCH SIMPLE
        ON UPDATE NO ACTION
        ON DELETE NO ACTION
)
"""

In [56]:
user_question = "Give me the details about the collegaues whoc joined in year 2015 and using macbook air"

In [57]:
prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt = prompt_template.format(colleague_data_schema=colleague_data_schema,
                                asset_data_schema=asset_data_schema,
                                question=user_question)

In [59]:
llm_model = ChatOpenAI(api_key=openai_api_key, temperature=0.5)
# response_text = llm_model.predict(prompt)
# response_text

In [66]:
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()

In [64]:
chain = llm_model | parser

In [68]:
print(chain.invoke(prompt))

Tables:
1. colleague_data
2. asset_data

Columns:
1. employee_id (from colleague_data)
2. emp_name (from colleague_data)
3. date_of_joining (from colleague_data)
4. laptop_model (from asset_data)
